## Task:
1. Try to quantize the full model, run on data, check the difference in the output wrt. the fp32 model.
2. Try to quantize the full model, run on data, check the difference in the output wrt. the fp16 model.
3. Try to quantize the full model, run on data, check the difference in the output wrt. the int8 model.
4. what matters is if you quantize a layer or the whole network, how does the inference runtime of the network change?
5. measuring the inference throughput (time per batch)? that would be the relevant metric, along with the error in the output compared to the fp32 version
6. run the inference on a bunch of real events from the dataset, and check the error introduced by quantization in the inference compared to fp32. and also measure the inference time

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import h5py

In [2]:
 '../../tensorflow_datasets/clic_edm_qq_pf/1.5.0/clic_edm_qq_pf-test.array_record-00000-of-00032'

In [7]:
import tensorflow as tf

# Define a function to parse a single TFRecord example
def parse_tfrecord_fn(example):
    # Define your features and their types
    feature_description = {
        'feature_name1': tf.io.FixedLenFeature([], tf.float32),
        'feature_name2': tf.io.FixedLenFeature([], tf.int64),
        # Add more features as needed
    }
    example = tf.io.parse_single_example(example, feature_description)
    return example

# Create a TFRecordDataset
dataset = tf.data.TFRecordDataset([ '../../tensorflow_datasets/clic_edm_qq_pf/1.5.0/clic_edm_qq_pf-test.array_record-00000-of-00032'])

# Apply the parsing function to each record
parsed_dataset = dataset.map(parse_tfrecord_fn)

# Iterate through the dataset
for record in parsed_dataset.take(1):
    print(record)

2023-10-23 11:28:32.026470: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-23 11:28:32.242476: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-23 11:28:32.242517: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-23 11:28:32.244068: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-23 11:28:32.331422: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-23 11:28:32.333183: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

DataLossError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} corrupted record at 0 (Is this even a TFRecord file?) [Op:IteratorGetNext] name: 